### Gera os gráficos das bases de dados coletadas

In [11]:
import pandas as pd
import os
import matplotlib.pyplot as plt
dataNames = [
    "PETR3_B_0_30min", 
    "WDOFUT_F_0_30min", 
    "WINFUT_F_0_30min",
]

setDivision = [0.1, 0.7, 0.2]              # Size of the [optimization, train, test] set
outputName  = "Fechamento"                 # Name of the output variable0

dir_name = f"../Results/datasPlots"
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

for dataName in dataNames:
    Y = pd.read_csv(f'../Data/Generated/{dataName}_OUT.csv', sep=";")['OutPut |T+1|']
    plt.figure(figsize=(10, 5))
    plt.plot(Y)
    plt.axvline(x=len(Y) * 0.1, color='red', linestyle='--')
    plt.axvline(x=len(Y) * 0.8, color='blue', linestyle='--')
    plt.text(len(Y) * -0.03, Y.max() * 1, 'Otimização', color='red')
    plt.text(len(Y) * 0.38, Y.max() * 1, 'Treinamento', color='black')
    plt.text(len(Y) * 0.90, Y.max() * 1, 'Teste', color='blue')
    plt.legend()
    # plt.title(f"Ativo {dataName.split('_')[0]}")
    plt.xticks([])
    plt.xlabel("Amostras")
    plt.ylabel("Valor (R$)")
    plt.savefig(f"{dir_name}/{dataName.split('_')[0]}_fechamento.png")
    plt.close()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Verifica os melhores modelos para cada base de dados

In [12]:
from models_classification import GetModelsClassification
from models_regression import GetModelsRegression

dataNames = [
    "PETR3_B_0_30min", 
    "PETR3_B_0_60min", 

    "WDOFUT_F_0_30min", 
    "WDOFUT_F_0_60min", 

    "WINFUT_F_0_30min",
    "WINFUT_F_0_60min",
]

for dataName in dataNames:
    SVM, KNN, LR = GetModelsClassification(dataName)        # Obtém os modelos de classificação
    LSTM, MLP, RNN = GetModelsRegression(dataName)          # Obtém os modelos de regressão

    print(f"-------- Modelos definidos para a base de dados {dataName} --------")
    print(f"Modelos de classificação: ")
    print(f"Modelo SVM: {SVM.get_params()}")
    print(f"Modelo KNN: {KNN.get_params()}")
    print(f"Modelo LR: {LR.get_params()}", end="\n\n")
    print(f"Modelos de regressão: ")
    print(f"Modelo MLP: {[camada.units for camada in MLP.layers]}")
    print(f"Modelo RNN: {[camada.units for camada in RNN.layers]}")
    print(f"Modelo LSTM: {[camada.units for camada in LSTM.layers]}")
    print(f"-------------------------------------------------------------------", end="\n\n")

-------- Modelos definidos para a base de dados PETR3_B_0_30min --------
Modelos de classificação: 
Modelo SVM: {'C': 15.01, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 1, 'decision_function_shape': 'ovr', 'degree': 4, 'gamma': 'scale', 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Modelo KNN: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 34, 'p': 1, 'weights': 'distance'}
Modelo LR: {'C': 10, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': False, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 50, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': True}

Modelos de regressão: 
Modelo MLP: [60, 60, 60, 28, 1]
Modelo RNN: [80, 80, 96, 64, 1]
Modelo LSTM: [20, 20, 20, 1]
-------------------------------

### Gera tabelas de comparação para modelos de regressão

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
import os

dataNames = [
    "PETR3_B_0_30min", 
    "WDOFUT_F_0_30min", 
    "WINFUT_F_0_30min",

    "PETR3_B_0_60min", 
    #"WDOFUT_F_0_60min", 
    "WINFUT_F_0_60min",
]

setDivision = [0.1, 0.7, 0.2]              # Size of the [optimization, train, test] set
outputName  = "Fechamento"                 # Name of the output variable0

dir_name = f"../Results/test/logs/regression"
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

for dataName in dataNames:
    res = pd.DataFrame(columns=['Model', 'MAE', 'RMSE'])
    datasRegression = pd.read_csv(f'../Results/test/regression/{dataName}_predictions.csv', sep=";")
    datasStatistics = pd.read_csv(f'../Results/test/statistic/{dataName}_predictions.csv', sep=";")
    outDatas = pd.read_csv(f'../Data/Cut/statistic/Y/Test_{dataName}.csv', sep=";")['OutPut |T+1|']
    datas = pd.concat([datasRegression, datasStatistics], axis=1)
    print("Data Name:       ", dataName)
    print("Datas Shape:     ", datas.shape)
    print("OutDatas Shape:  ", outDatas.shape)
    for colum in datas.columns:
        mae = mean_absolute_error(outDatas, datas[colum])
        rmse = np.sqrt(mean_squared_error(outDatas, datas[colum]))
        res = pd.concat([res, pd.DataFrame([[colum, mae, rmse]], columns=['Model', 'MAE', 'RMSE'])], axis=0)
    res.to_csv(f"{dir_name}/{dataName}.csv", sep=";", index=False)

Data Name:        PETR3_B_0_30min
Datas Shape:      (1342, 6)
OutDatas Shape:   (1342,)
Data Name:        WDOFUT_F_0_30min
Datas Shape:      (1610, 6)
OutDatas Shape:   (1610,)
Data Name:        WINFUT_F_0_30min
Datas Shape:      (1610, 6)
OutDatas Shape:   (1610,)
Data Name:        PETR3_B_0_60min
Datas Shape:      (683, 6)
OutDatas Shape:   (683,)
Data Name:        WINFUT_F_0_60min
Datas Shape:      (802, 6)
OutDatas Shape:   (802,)


## Exibe logs para modelos treinados

In [14]:
import os 
import pandas as pd
# print("Operations History:        \n", operationsHistory.to_string(index=False))

path = "../Results/test/logs/class"
files = os.listdir(path)
for file in files:
    if file.endswith(".csv"):
        print(f"================================================ {file} ===========================================================")
        df = pd.read_csv(path+"/"+file, sep=";")
        print(df.to_string(index=False), end="\n\n")


================================================ PETR3_B_0_15min_logs.csv ===========================================================
                  model  qtdBuying  inirialValue  finalValue  percentual  accuracy       f1  truePositives  trueNegatives  falsePositives  falseNegatives
                    SVM        456          1000  816.853070  -18.314693  0.643560 0.693516          513.0          861.0           368.0           393.0
                    KNN        475          1000  771.552742  -22.844726  0.633255 0.673342          545.0          807.0           336.0           447.0
                     LR        469          1000  830.756976  -16.924302  0.641218 0.672370          583.0          786.0           298.0           468.0
                   LSTM          1          1000  812.396694  -18.760331  0.587822 0.740260            1.0         1254.0           880.0             0.0
                    MLP          1          1000  812.396694  -18.760331  0.587822 0.740260     

### Exibe logs de regressão

In [15]:
import os 
import pandas as pd
# print("Operations History:        \n", operationsHistory.to_string(index=False))

path = "../Results/test/logs/regression"
files = os.listdir(path)
for file in files:
    if file.endswith(".csv"):
        print(f"================================================ {file} ===========================================================")
        df = pd.read_csv(path+"/"+file, sep=";")
        print(df.to_string(index=False), end="\n\n")

================================================ PETR3_B_0_60min.csv ===========================================================
 Model      MAE     RMSE
  LSTM 4.348124 4.570121
   MLP 4.723340 5.183967
   RNN 4.340650 4.562272
 ARIMA 0.079205 0.114771
SARIMA 0.079692 0.114520
 GARCH 0.604997 0.836969

================================================ WDOFUT_F_0_30min.csv ===========================================================
 Model         MAE        RMSE
  LSTM 4957.349252 4959.809258
   MLP  440.029471  472.735178
   RNN 4062.172952 4065.174702
 ARIMA    9.663642   13.786633
SARIMA    9.837974   13.986968
 GARCH   50.555641   64.577240

================================================ PETR3_B_0_30min.csv ===========================================================
 Model      MAE     RMSE
  LSTM 4.346582 4.588245
   MLP 6.119167 6.353164
   RNN 4.516052 4.747392
 ARIMA 0.055946 0.080992
SARIMA 0.057220 0.081907
 GARCH 0.418511 0.594305

==========================================

## Exibe logs para estratégias implementadas

In [16]:
import os 
import pandas as pd
# print("Operations History:        \n", operationsHistory.to_string(index=False))

path = "../Results/test/logs/economic"
files = os.listdir(path)
for file in files:
    if file.endswith(".csv"):
        print(f"================================================ {file} ===========================================================")
        df = pd.read_csv(path+"/"+file, sep=";")
        print(df.to_string(index=False), end="\n\n")

================================================ ensamble2_buying.csv ===========================================================
                         model  qtdBuying  inirialValue  finalValue  percentual
    strat_TCC_WINFUT_F_0_30min        397          1000 1180.377631   18.037763
strat_hurwicz_WINFUT_F_0_30min          1          1000 1181.388929   18.138893
                  Buy and Hold          1          1000 1178.706030   17.870603
     strat_TCC_PETR4_B_0_30min         48          1000 1347.181726   34.718173
 strat_hurwicz_PETR4_B_0_30min          1          1000 1306.487696   30.648770
                  Buy and Hold          1          1000 1310.886644   31.088664
      strat_TCC_PETR3_B_0_5min        332          1000  823.566918  -17.643308
  strat_hurwicz_PETR3_B_0_5min          1          1000  856.312292  -14.368771
                  Buy and Hold          1          1000  856.312292  -14.368771
     strat_TCC_PETR3_B_0_15min        277          1000  739.905818  -